In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(r'C:\Work_Directory\Learn\DS_Projects\Diamond_Price_Prediction\notebooks\data\gemstone.csv')
data.head(10)
data.drop(columns='id',axis=1,inplace=True)

[1.52 'Premium' 'F' 'VS2' 62.2 58.0 7.27 7.33 4.55 13619]


In [3]:
# Independent and Dependent Features
X = data.drop(labels='price',axis=1)
Y = data['price']

In [4]:
categorical_features = X.select_dtypes(include='object').columns
numerical_features = X.select_dtypes(exclude='object').columns
print(categorical_features,numerical_features)

Index(['cut', 'color', 'clarity'], dtype='object') Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')


#### Custom ranking for each of the ordinal variable
- Pipeline will be used
- Based on the ranking, a list is created to map the values.

In [5]:
cut_map = ['Premium','Very Good','Ideal','Good','Fair']
cut_categories  = cut_map[::-1]
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]

### Applying Pipeline
- Simple Imputer - Used for handling missing values. options can be given to choose mean value etc.
- Scaling - standard scaling is used for scaling the data.
- Ordinal Encoded - used for encoding the categorical features.

In [6]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

- Pipeline

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#### Numerical Pipeline

In [8]:
numerical_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

#### Categorical Pipeline

In [9]:
categorical_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler', StandardScaler())
    ]
)

#### Column Transformer
The ColumnTransformer in scikit-learn is essentially a way to apply different preprocessing steps to different subsets of columns in a dataset. The math behind it involves the individual transformations you apply to each subset of columns. Let's break down the math involved in some common transformations:
- Standard Scaling - Standard scaling transforms your data to have a mean of 0 and a standard deviation of 1.
- One-hot encoding converts categorical values into a binary vector representation. If a feature has k unique categories, it is transformed into k binary features.
- Imputation involves filling missing values.Common imputations are mean,median and mode.

Concatenating transformed numerical and categorical features back into a single matrix involves a bit of linear algebra and matrix operations. The idea is to combine multiple matrices (or arrays) side-by-side (horizontally) so that each row in the resulting matrix corresponds to a single data point in the original dataset.

In [10]:
preprocessor=ColumnTransformer([
    ('numerical_pipeline',numerical_pipeline,numerical_features),
    ('categorical_pipeline',categorical_pipeline,categorical_features)
])

### Model Training

#### Train Test Split

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [12]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [13]:
X_train.head()

,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,-0.652673,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-1.687566,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,1.417114,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,1.417114,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,-0.652673,2.144670,1.352731


#### Model Training

In [14]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [15]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [16]:
regression.coef_

array([ 6434.22959729,  -154.00568244,  -123.98485273, -1691.9243623 ,
        -530.7007583 ,   -61.41330294,    34.68628051,  -460.94771592,
         656.37286862])

In [17]:
regression.intercept_

np.float64(3970.7662895476774)

In [18]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

#### Train Multiple models

In [19]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'DecisionTreeRegressor':DecisionTreeRegressor(),
    'RandomForestRegressor':RandomForestRegressor()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1014.8135235813573
MAE: 674.729401097174
R2 score 93.67776382515095


Lasso
Model Training Performance
RMSE: 1014.7839731646295
MAE: 675.813110767269
R2 score 93.67813201494975


Ridge
Model Training Performance
RMSE: 1014.8145261557182
MAE: 674.759592466091
R2 score 93.67775133317113


Elasticnet
Model Training Performance
RMSE: 1534.864252047287
MAE: 1060.342144733142
R2 score 85.537673373368


DecisionTreeRegressor
Model Training Performance
RMSE: 831.7873312385402
MAE: 423.21988221518114
R2 score 95.75260329164841


RandomForestRegressor
Model Training Performance
RMSE: 609.8948013143173
MAE: 310.9930560821137
R2 score 97.7164628244091


